In [ ]:
from pyrosm import OSM
import pandas as pd
import geopandas as gpd

It is proving very difficult to filter trails from OSM without getting far too little or far too much. I think I will need to get far too much, and then exclude things once it is a pandas data frame. The other option is to try filtering by what is in the "JSON" column, though I think I am partially already doing that. Another approach would be to do lots of narrow searches and add them all into the same gdf. That might be the easiest. 

In [ ]:
pbf_path = "data/trails/south-carolina-latest.osm.pbf"

trail_filter_mtb = {"route":["mtb"]}

osm = OSM(pbf_path) #use get_data function to pull directly from geofabrik

trails_mtb = osm.get_data_by_custom_criteria(
    custom_filter=trail_filter_mtb,
    filter_type="keep",
)

In [ ]:
trails_mtb_2 = osm.get_data_by_custom_criteria(
    custom_filter={},
    osm_keys_to_keep=["mtb:scale"],
    filter_type="keep",
)

In [ ]:
trail_filter_hiking_1 = {}

trails_hiking_1 = osm.get_data_by_custom_criteria(
    osm_keys_to_keep=["sac_scale", "osmc:symbol"],
    custom_filter=trail_filter_hiking_1,
    filter_type="keep",
)

In [ ]:
trail_filter_hiking_2 = {"route":["hiking"]}

trails_hiking_2 = osm.get_data_by_custom_criteria(
    custom_filter=trail_filter_hiking_2,
    filter_type="keep",)

In [ ]:
trails_mtb_combined = pd.concat([trails_mtb, trails_mtb_2], ignore_index=True)
# trails_mtb_combined.head()

In [ ]:
trails_mtb_combined.drop_duplicates()

In [ ]:
trails_hiking_combined = pd.concat([trails_hiking_1, trails_hiking_2], ignore_index=True)
# trails_hiking_combined.head()

In [ ]:
all_trails = pd.concat([trails_hiking_combined, trails_mtb_combined], ignore_index=True)
all_trails.drop_duplicates()
# all_trails.head()

In [ ]:
all_trails['tags']

In [ ]:

def trail_name(tags):
    name = tags.get('name')

In [ ]:
m = all_trails.explore()

m

This method of stacking different filters into one geodataframe seemed to give the best results. I'll move forward with this. 

In [ ]:
all_trails_5070 = all_trails.to_crs(epsg=5070) #reprojecting to the same as the xarray datasets. Should be reprojected before doing the buffer. 

In [ ]:
all_trails_buffered = all_trails_5070.buffer(200) #200 meter buffer. Returns polygons. 
all_trails_buffered

In [ ]:
all_trails['buffered'] = all_trails_buffered
sc_trails_original = all_trails[['id', 'tags', 'osm_type', 'geometry']]
sc_trails_original.to_file("data/trails/sc_trails_original.gpkg", driver="GPKG")

In [ ]:
sc_trails_buffered = all_trails[['id', 'tags', 'osm_type', 'buffered']]
sc_trails_buffered.set_geometry("buffered", inplace=True)
sc_trails_buffered.to_file("data/trails/sc_trails_buffered.gpkg", driver="GPKG")

In [ ]:
sc_trails_buffered

In [ ]:
m = sc_trails_buffered.explore()

m

Perform a unary union or dissolve the overlapping polygons. Then I have to figure out how to tag that polygon (or keep original attributes) based on the trail/trail network name. Need a function that can pull the name of the trail out of the tags column, then group them somehow, or determine which has the trail network name. 

First layer - Pull Trail name from tags if it is there. 
Second layer - Use land use (or whatever is on the basemap ie Paris Mountain State Park polygon) to give buffer polygons a label. Then could list all trail names within that area. Or click to get individual trails. 
